## 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import  DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [ ]:
train_tfm = transforms.Compose([
    # 对图片进行相同的预处理
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),   #随机水平翻转
    transforms.RandomRotation(degrees=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 测试集不需要进行数据增广，只需要放缩与normalize
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
batch_size = 64 # 可根据自己显存进行更改
# 构建数据集
train_set = ImageFolder("./data/train",  transform=train_tfm)
test_set = ImageFolder("./data/test",  transform=test_tfm)
num_to_class = train_set.classes #记录数字到标签的映射
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, shuffle=False)

## 构建模型

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights #导入预训练模型

In [ ]:
class Classifier(nn.Module):    #超参数可自己修改
    def __init__(self):
        super(Classifier, self).__init__()
        self.cnn_layers = resnet18(weights=ResNet18_Weights)
        self.fc_layers = nn.Sequential(
            nn.Linear(1000, 512),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 5)
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.fc_layers(x)
        return x

## 训练与预测

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
n_epochs = 3  
# 初始化模型，并将其放在指定的设备上.
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9,
                              weight_decay=1e-5) 
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    model.train()
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):
        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

In [ ]:
# ---------- Testing ----------
model.eval()
test_loss = []
test_accs = []
for batch in tqdm(test_loader):
    imgs, labels = batch
    with torch.no_grad():
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))
            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            # Record the loss and accuracy.
            test_loss.append(loss.item())
            test_accs.append(acc)
    test_loss = sum(test_loss) / len(test_loss)
    test_acc = sum(test_accs) / len(test_accs)
    print(f"[ test | {epoch + 1:03d}/{n_epochs:03d} ] loss = {test_loss:.5f}, acc = {test_acc:.5f}")

## 保存模型并加载

In [ ]:
torch.save(model.state_dict(), 'classify_nsfw_v1.pth')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
num_to_class = ['drawings', 'hentai', 'neutral', 'porn', 'sexy']
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.cnn_layers = resnet18(weights=ResNet18_Weights)
        self.fc_layers = nn.Sequential(
            nn.Linear(1000, 512),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 5)
        )
    def forward(self, x):

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:
clone = Classifier()
clone.load_state_dict(torch.load('classify_nsfw_v1.pth'))
clone.eval()
images = os.listdir('./raw_data/')  #替换成自己测试图片的文件夹，下同
for image in images:
    exp = Image.open(f'./raw_data/{image}')
    inp = test_tfm(exp).unsqueeze(0)
    with torch.no_grad():
        print(image)
        prediction = torch.nn.functional.softmax(clone(inp)[0], dim=0)
        index = prediction.argmax(dim=0)
        for i in range(5):
            if prediction[i] > 0.1:
                print(f'{num_to_class[i]} 置信度:{prediction[i]:.4f}')